<a href="https://colab.research.google.com/github/ANDNX/AIchatbot/blob/main/Mainproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import zipfile
import os
zip_path = '/content/ml-latest-small (1).zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_files')

extracted_files = os.listdir('/content/extracted_files')
print("Extracted files:", extracted_files)


Extracted files: ['ml-latest-small']


In [ ]:
import pandas as pd

movies_df = pd.read_csv('/content/extracted_files/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('/content/extracted_files/ml-latest-small/ratings.csv')

In [ ]:
pip install imdbpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
from imdb import IMDb, IMDbDataAccessError

ia = IMDb()

def fetch_imdb_data(movie_title):
    try:
        search_results = ia.search_movie(movie_title)  # Search for the movie
        if search_results:  # Check if search results are not empty
            movie_id = search_results[0].movieID
            movie = ia.get_movie(movie_id)
            description = movie.get('plot', [""])[0]
            cover_url = movie.get('full-size cover url')
            return description, cover_url
        else:
            return None, None  # Return None if no results found
    except IMDbDataAccessError:
        return None, None

# Keep only the first 1000 rows
movies_df = movies_df.head(10)

# Apply the fetch_imdb_data function to the 'title' column in movies_df
movies_df['description'], movies_df['cover_url'] = zip(*movies_df['title'].apply(fetch_imdb_data))
ratings_df = ratings_df.head(1000)
# Print the first few rows to check the results
print(movies_df.head())


2024-11-13 07:25:32,700 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0114709/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 500: ''>},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 525, in open
    response = meth(req, response)
  File "/usr/lib/python3.10/urllib/request.py", line 634, in http_response
    response = self.parent.error(
  File "/usr/lib/python3.10/urllib/request.py", line 563, in error
    return self._call_chain(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 643, in http_error_

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres description cover_url  
0  Adventure|Animation|Children|Comedy|Fantasy        None      None  
1                   Adventure|Children|Fantasy        None      None  
2                               Comedy|Romance        None      None  
3                         Comedy|Drama|Romance        None      None  
4                                       Comedy        None      None  


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()
movies_df['text_embedding'] = movies_df['description'].apply(lambda x: get_text_embedding(str(x)) if x else None)


In [ ]:
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

image_model = models.resnet50(pretrained=True)
image_model.eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_image_embedding(url):
    if url:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img = preprocess(img).unsqueeze(0)
        with torch.no_grad():
            features = image_model(img)
        return features.numpy()
    return None

movies_df['image_embedding'] = movies_df['cover_url'].apply(get_image_embedding)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
movies_df = movies_df.head(10)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

image_model = models.resnet50(pretrained=True)
image_model.eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_image_embedding(url):
    if url:
        try:
            response = requests.get(url)
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img = preprocess(img).unsqueeze(0)
            with torch.no_grad():
                features = image_model(img)
            return features.flatten().numpy()
        except Exception as e:
            print(f"Error fetching or processing image: {e}")
            return np.zeros(2048)
    return np.zeros(2048)

def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)

    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

movies_df['text_embedding'] = movies_df['title'].apply(get_text_embedding)
movies_df['image_embedding'] = movies_df['cover_url'].apply(get_image_embedding)

X_text = np.array(movies_df['text_embedding'].tolist())
X_image = np.array(movies_df['image_embedding'].tolist())
X_combined = np.concatenate((X_text, X_image), axis=1)
def build_model(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(input_layer)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='linear')(x)
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

input_dim = X_combined.shape[1]
model = build_model(input_dim)
y = np.random.random(size=(X_combined.shape[0],))
model.fit(X_combined, y, epochs=5, batch_size=32, validation_split=0.2)
print(movies_df[['title', 'text_embedding', 'image_embedding']].head())

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.3038 - val_loss: 0.1395
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.1301 - val_loss: 0.1258
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1264 - val_loss: 0.0466
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0858 - val_loss: 0.0131
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0820 - val_loss: 0.0113
                                title  \
0                    Toy Story (1995)   
1                      Jumanji (1995)   
2             Grumpier Old Men (1995)   
3            Waiting to Exhale (1995)   
4  Father of the Bride Part II (1995)   

                                      text_embedding  \
0  [-0.479142, -0.33330202, -0.46235853, -0.47733...   
1  [-0.5604112, -0.48207977, -0.34104246, -0.4804...   
2  [-0.36912534, -0.0023814728, -0.36450836, -0.4...   
3  [-0.2735409, -0.069171384, -0.36437643, -0.336...   
4  [-0.70503134, -0.21561272, -0.15182748, -0.457...   

        

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score
X_train, X_val, y_train, y_val = train_test_split(X_combined, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=5, batch_size=2, validation_data=(X_val, y_val))
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error (MSE) on validation set: {mse}")
absolute_error = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error (MAE) on validation set: {absolute_error}")

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1239 - val_loss: 0.1024
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0323 - val_loss: 0.0451
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0411 - val_loss: 0.1272
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0438 - val_loss: 0.0503
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0208 - val_loss: 0.1874
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Mean Squared Error (MSE) on validation set: 0.187350086862091
Mean Absolute Error (MAE) on validation set: 0.42880018705386747


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_new_movie_embedding(title, cover_url):
    text_embedding = get_text_embedding(title)


    image_embedding = get_image_embedding(cover_url)


    new_movie_embedding = np.concatenate((text_embedding.flatten(), image_embedding.flatten()), axis=0)

    return new_movie_embedding
chosen_movie_index = 2
chosen_movie = movies_df.iloc[chosen_movie_index]
chosen_movie_embedding = get_new_movie_embedding(chosen_movie['title'], chosen_movie['cover_url'])
all_movie_embeddings = movies_df.apply(
    lambda row: get_new_movie_embedding(row['title'], row['cover_url']), axis=1
)
all_movie_embeddings = np.array(all_movie_embeddings.tolist())
similarities = cosine_similarity([chosen_movie_embedding], all_movie_embeddings)[0]
movies_df['similarity'] = similarities

recommended_movies = movies_df.sort_values(by='similarity', ascending=False).drop(index=chosen_movie_index)
top_recommended_movies = recommended_movies.head(5)
print(f"Movies Recommended Based on '{chosen_movie['title']}':")
print(top_recommended_movies[['title', 'similarity']])


Movies Recommended Based on 'Grumpier Old Men (1995)':
                                title  similarity
8                 Sudden Death (1995)    0.977542
7                 Tom and Huck (1995)    0.975056
4  Father of the Bride Part II (1995)    0.973370
3            Waiting to Exhale (1995)    0.972946
9                    GoldenEye (1995)    0.971611


In [ ]:
relevant_items = ['Tom and Huck (1995)', 'Father of the Bride Part II (1995)']

def calculate_mrr(recommended_list, relevant_items):

    for rank, movie in enumerate(recommended_list, start=1):
        if movie in relevant_items:
            return 1.0 / rank
    return 0.0
recommended_titles = [
    "Tom and Huck (1995)",
    "Father of the Bride Part II (1995)",
    "Grumpier Old Men (1995)",
    "GoldenEye (1995)"
]

mrr_score = calculate_mrr(recommended_titles, relevant_items)
print("MRR:", mrr_score)


MRR: 1.0
